In [ ]:
import timeit

#for testing in Colab
#import os
#os.chdir('/content/drive/MyDrive/Диплом /GEFEST_impl/gefest_main')



Setting up the environment

In [ ]:
! pip install -r requirements.txt 

Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment
Ignoring matplotlib: markers 'python_version == "3.8"' don't match your environment


Setting up field, well, road and simulation parameters for experiments

In [ ]:
import configparser
import numpy as np

from read_data import read_data
from structures.field import impossible_points_generation, Field
from structures.well import Well
from structures.simulation import simulation
from optimize_wells import Optimization_alg
from functions import NPV

config = configparser.ConfigParser()
config.read('config.ini')

field_parameters = config['Field Parameters']
road_parameters = config['Road Parameters']
well_parameters = config['Well Parameters']
simulation_parameters = config['Simulation Parameters']

data_path = field_parameters.get('data_path', fallback = './data/spe_phi.dat')

count_impossible_points = field_parameters.getint('count_impossible_points', fallback = 5)

deposit_size_x = field_parameters.getint('size_x', fallback = 60)
deposit_size_y = field_parameters.getint('size_y', fallback = 220)
deposit_size_z = field_parameters.getint('size_z', fallback = 85)
deposit_size = (deposit_size_x, deposit_size_y, deposit_size_z)

size_x = field_parameters.getint('field_size_x', fallback = 20)
size_y = field_parameters.getint('field_size_y', fallback = 20)
size_z = field_parameters.getint('field_size_z', fallback = 20)
field_size = (size_x, size_y, size_z)

r = well_parameters.getint('r', fallback = 2)

T = simulation_parameters.getint('T', fallback = 10)
production_volume = simulation_parameters.getint('v', fallback = 40)
price_oil = simulation_parameters.getint('price_oil', fallback = 70)

data = read_data(data_path, deposit_size, field_size)




The data is correctly read and processed. Deposit size: (60, 220, 85). Size of the cut field: (20, 20, 20).


Creating and saving random maps of inaccessible points for experiments with 2, 4, 6, 10, 20%.

In [ ]:
import timeit

import matplotlib.pyplot as plt

from gefest.core.geometry.geometry_2d import Geometry2D
from gefest.core.opt.analytics import EvoAnalytics
from gefest.core.opt.optimize import optimize
from gefest.core.opt.setup import Setup
from gefest.core.structure.domain import Domain
from gefest.core.structure.structure import Structure, distance, get_random_poly
from gefest.core.structure.point import Point
from gefest.core.viz.struct_vizualizer import StructVizualizer
from gefest.core.structure.polygon import Polygon

import numpy as np

def create_invalid_points(rate = 0.05):
  invalid_points = [[],[]]
  is_closed = True
  geometry = Geometry2D(is_closed=is_closed)
  domain = Domain(allowed_area=[(0, 0),
                                (0, 20),
                                (20, 20),
                                (20, 0),
                                (0, 0)],
                  geometry=geometry,
                  max_poly_num=1,
                  min_poly_num=1,
                  max_points_num=10,
                  min_points_num=3,
                  is_closed=is_closed)
  
  
  count_ = 400 * rate
  counter_ = 0 
  structure = Structure([])
  
  while counter_ < count_:
    poly = get_random_poly(structure, domain, invalid_points = invalid_points)


    for i in range(20, 0, -1):
      for j in range(20):
        if geometry.is_contain_point(poly, Point(i, j)) and counter_ < count_:
          invalid_points[0].append(i)
          invalid_points[1].append(j)
          counter_ += 1

  print(f'Created invalid_points: {rate}, {len(invalid_points[0])}')
  return invalid_points
invalid_areas_2 =[[[6, 6, 14, 13, 13, 12, 12, 12], [6, 7, 13, 13, 14, 14, 15, 16]],
                [[14, 14, 14, 13, 13, 13, 13, 13], [6, 7, 8, 4, 5, 6, 7, 8]],
                [[7, 7, 6, 6, 6, 6, 6, 5], [10, 11, 7, 8, 9, 10, 11, 7]]]
invalid_areas_4 = [[[16, 15, 15, 14, 14, 14, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18], [1, 1, 2, 1, 2, 3, 4, 5, 6, 7, 8, 9, 6, 7, 8, 9]],
                   [[3, 3, 3, 3, 3, 2, 2, 2, 1, 3, 19, 19, 19, 19, 18, 18], [16, 17, 18, 19, 15, 18, 19, 15, 16, 18, 2, 3, 4, 5, 2, 3]],
                   [[10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 12, 12, 1, 2, 2, 2], [6, 7, 8, 9, 10, 11, 8, 9, 10, 7, 8, 9, 16, 15, 16, 17]]]
invalid_areas_6 = [[[7, 7, 7, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4], [14, 15, 16, 11, 12, 13, 14, 15, 16, 9, 10, 11, 12, 13, 14, 15, 8, 9, 10, 11, 12, 13, 14, 15]],
                   [[10, 10, 9, 9, 9, 9, 8, 8, 8, 8, 7, 7, 7, 7, 7, 6, 6, 6, 6, 5, 5, 5, 5, 4], [17, 18, 16, 17, 18, 19, 16, 17, 18, 19, 16, 17, 18, 19, 15, 15, 16, 17, 18, 15, 16, 17, 18, 19]],
                   [[4, 4, 4, 4, 13, 13, 13, 13, 13, 13, 13, 13, 12, 12, 12, 12, 12, 12, 12, 11, 11, 11, 11, 11], [12, 13, 14, 15, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 3, 4, 5, 6, 7]]]
invalid_areas_10 = [[[13, 13, 13, 13, 12, 12, 12, 12, 12, 12, 12, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9], [16, 17, 18, 19, 13, 14, 15, 16, 17, 18, 19, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 8, 9, 10, 11, 12, 13]],
                    [[12, 11, 11, 10, 10, 10, 9, 9, 9, 9, 9, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4], [2, 2, 3, 2, 3, 4, 1, 2, 3, 4, 5, 1, 2, 3, 4, 1, 2, 3, 17, 18, 19, 14, 15, 16, 17, 18, 19, 13, 14, 15, 16, 17, 18, 19, 14, 15, 16, 17, 18, 19]],
                    [[19, 9, 9, 9, 8, 8, 7, 13, 12, 12, 11, 11, 11, 11, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 6], [9, 13, 14, 15, 14, 15, 15, 15, 14, 15, 13, 14, 15, 16, 12, 13, 14, 15, 16, 11, 12, 13, 14, 15, 16, 17, 11, 12, 13, 14, 15, 16, 17, 12, 13, 14, 15, 16, 17, 13]]]                
invalid_areas_20 = [[[18, 18, 18, 10, 10, 10, 10, 10, 10, 14, 13, 12, 12, 12, 11, 11, 11, 10, 10, 10, 10, 9, 9, 9, 9, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 14, 13, 13, 12, 12, 12, 11, 18, 18, 17, 17, 16, 18, 18, 18, 18, 17, 17, 17, 16, 16, 15, 7, 6, 5, 4, 16, 15, 15, 14, 14, 13, 13, 13, 12, 12, 12, 11, 11], [12, 13, 14, 9, 10, 11, 12, 13, 14, 5, 5, 4, 5, 6, 4, 5, 6, 3, 4, 5, 6, 3, 4, 5, 6, 2, 3, 4, 5, 6, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 1, 2, 1, 2, 3, 1, 1, 2, 1, 2, 1, 5, 6, 7, 8, 5, 6, 7, 5, 6, 5, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 3, 1, 2, 3, 1, 2]],
                    [[10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 9, 9, 9, 9, 9, 8, 8, 8, 8, 7, 7, 7, 6, 6, 5, 10, 9, 9, 8, 8, 8, 7, 7, 7, 6, 6, 11, 11, 11, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 8], [3, 4, 5, 6, 7, 3, 4, 5, 6, 7, 8, 2, 3, 4, 5, 6, 7, 8, 2, 3, 4, 5, 6, 7, 8, 2, 3, 4, 5, 6, 7, 8, 3, 4, 5, 6, 14, 15, 16, 17, 18, 15, 16, 17, 18, 16, 17, 18, 17, 18, 18, 17, 16, 17, 15, 16, 17, 15, 16, 17, 16, 17, 12, 13, 14, 12, 13, 14, 15, 16, 17, 12, 13, 14, 15, 16, 17, 18, 19, 12]],
                    [[9, 8, 8, 7, 7, 7, 14, 14, 14, 14, 13, 13, 11, 19, 19, 13, 13, 12, 12, 12, 12, 11, 11, 11, 11, 12, 11, 11, 11, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4], [3, 2, 3, 1, 2, 3, 2, 3, 4, 5, 3, 4, 6, 4, 5, 12, 13, 11, 12, 13, 14, 11, 12, 13, 14, 19, 17, 18, 19, 16, 17, 18, 19, 14, 15, 16, 17, 18, 19, 13, 14, 15, 16, 17, 18, 19, 12, 13, 14, 15, 16, 17, 18, 19, 12, 13, 14, 15, 16, 17, 18, 19, 11, 12, 13, 14, 15, 16, 17, 18, 19, 11, 12, 13, 14, 15, 16, 17, 18, 19]]]
 

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


General criterion for comparing algorithms

In [ ]:
def npv_joint(struct: Structure, start_point = (0, 0), end_point = (20, 20), coord_wells = [[], []], npv_f = None, wells = None, invalid_points = [[], []]):
    npv_value = 0

    l = []
    p = []
    for poly in struct.polygons:
        l.append(geometry.get_length(poly))
        p.append(poly.points)
    length = sum(l)
    
    ratio = length 
    ratio += dist_to_extreme_point(start_point, p[0][0]) + dist_to_extreme_point(end_point, p[0][-1])
    for i in range(len(invalid_points[0])):
        if distance(Point(invalid_points[0][i], invalid_points[1][i]), struct, geometry) <= 0.5:
            ratio += 4
    npv_value = - ratio * 3000 - npv_f.calculated_NPV(wells, struct)

    return npv_value

NPV for a naive approach

In [ ]:
def npv_through_wells(coord_wells = [[], []], start_point = (0, 0), end_point = (20, 20), wells = None):
    points = []
    points.append(Point(*start_point))
    arg = np.argsort(coord_wells[0])
    coord_wells[0] = np.array(coord_wells[0])[arg]
    coord_wells[1] = np.array(coord_wells[1])[arg]
    for i in range(coord_wells[0].shape[0]):
        points.append(Point(coord_wells[0][i], coord_wells[1][i]))
    points.append(Point(*end_point))

    poly = Polygon('naive_help', points=points)
    npv_value = - npv_f.calculated_NPV(wells) - geometry.get_length(poly) * 3000  

    return npv_value


General experiment generator

In [ ]:
def exp_with_rate(count_well = 5, rate = [0.00], rate_example = None, count_run = 5, count_run_rate = 5, joint = True, pso = False):
  npv_joint_mean = []
  npv_joint_mean_notjoint = []
  
  for rate_ in rate:
    print(f'Starting exp with rate: {rate_}')
    npv_joint_mean_rate = []
    npv_joint_mean_rate_notjoint = []
    for i in range(count_run_rate):
      step = 10
      if joint:
        step = 20
      print(f'Exp with rate: {rate_}, (No. {i} of {count_run_rate}), joint: {joint}')
      if rate_example is None: 
          invalid_points = create_invalid_points(rate_)
      else:
          invalid_points = rate_example[i]
      if not pso:
          optimized_well_, optimized_road_, history, history_road = algo(joint = joint, count_well = count_well, count_run = count_run, 
                                                                        invalid_points = invalid_points)
      else:
          optimized_well_, optimized_road_, history, history_road = algo_PSO(joint = joint, count_well = count_well, count_run = count_run, 
                                                                        invalid_points = invalid_points)

      history_boxplot = [[history[j][i] for j in range(len(history))] for i in range(len(history[0]))]
      
      flierprops = dict(marker='.', 
                  linestyle='none')
      
      plt.boxplot(history_boxplot, meanline = True, flierprops = flierprops)
      plt.plot(range(1, len(history_boxplot) + 1), np.mean(history_boxplot, axis = 1), label = 'Average NPV value for 5 runs')
      plt.xticks(range(1, len(history_boxplot) + 1, 2), [step * i * 2 + step for i in range(len(history_boxplot))], rotation=90)
      plt.xlabel('Iteration number')
      plt.ylabel('NPV values')
      plt.legend()
      plt.title(f'Learning curve for optimizing the location of {count_well} wells \n with considering and optimizing roads')
      plt.show()

      history_boxplot_road = [[history_road[j][i] for j in range(len(history_road))] for i in range(len(history_road[0]))]

      flierprops = dict(marker='.', 
                  linestyle='none')
      plt.boxplot(history_boxplot_road, meanline = True, flierprops = flierprops)
      plt.plot(range(1, len(history_boxplot_road) + 1), np.mean(history_boxplot_road, axis = 1), label = 'Average NPV roads value for 5 runs')
      plt.xticks(range(1, len(history_boxplot_road) + 1), [step * i + step for i in range(len(history_boxplot_road))], rotation=90)
      plt.xlabel('Iteration number')
      plt.ylabel('NPV roads values')
      plt.legend()
      plt.title(f'Learning curve for optimizing the location of roads \n with considering and optimizing {count_well} wells')
      plt.show()

      npv_joint_mean_rate.append(np.mean([npv_joint(optimized_road_[i], npv_f = npv_f, 
                                                    wells = optimized_well_[i], invalid_points = invalid_points) for i in range(count_run)]))
      print(f'{joint}:', npv_joint_mean_rate)

      print(f'Exp with rate: {rate_}, (No. {i} of {count_run_rate}), joint: {not joint}')
      step = 10
      if not joint:
        step = 20
      if not pso:
          optimized_well_notjoint, optimized_road_notjoint, history_notjoint, history_road_notjoint = algo(joint = not joint, 
                                                                                                          count_well = count_well, 
                                                                                                          count_run = count_run, 
                                                                                                          invalid_points = invalid_points)
      else: 
          optimized_well_notjoint, optimized_road_notjoint, history_notjoint, history_road_notjoint = algo_PSO(joint = not joint, 
                                                                                                          count_well = count_well, 
                                                                                                          count_run = count_run, 
                                                                                                          invalid_points = invalid_points)

      history_boxplot_notjoint = [[history_notjoint[j][i] for j in range(len(history_notjoint))] for i in range(len(history_notjoint[0]))]

      flierprops = dict(marker='.', 
                  linestyle='none')
      
      plt.boxplot(history_boxplot_notjoint, meanline = True, flierprops = flierprops)
      plt.plot(range(1, len(history_boxplot_notjoint) + 1), np.mean(history_boxplot_notjoint, axis = 1), 
               label = 'Average NPV value for 5 runs')
      plt.xticks(range(1, len(history_boxplot_notjoint) + 1, 2), [step * i * 2 + step for i in range(len(history_boxplot_notjoint))], rotation=90)
      plt.xlabel('Iteration number')
      plt.ylabel('NPV values')
      plt.legend()
      plt.title(f'Learning curve for optimizing the location of {count_well} wells \n with considering and optimizing roads')
      plt.show()

      history_boxplot_road_notjoint = [[history_road_notjoint[j][i] for j in range(len(history_road_notjoint))] 
                                       for i in range(len(history_road_notjoint[0]))]

      flierprops = dict(marker='.', 
                  linestyle='none')
      plt.boxplot(history_boxplot_road_notjoint, meanline = True, flierprops = flierprops)
      plt.plot(range(1, len(history_boxplot_road_notjoint) + 1), 
               np.mean(history_boxplot_road_notjoint, axis = 1), label = 'Average NPV roads value for 5 runs')
      plt.xticks(range(1, len(history_boxplot_road_notjoint) + 1), [step * i + step for i in range(len(history_boxplot_road_notjoint))], 
                 rotation=90)
      plt.xlabel('Iteration number')
      plt.ylabel('NPV roads values')
      plt.legend()
      plt.title(f'Learning curve for optimizing the location of roads \n with considering and optimizing {count_well} wells')
      plt.show()

      npv_joint_mean_rate_notjoint.append(np.mean([npv_joint(optimized_road_notjoint[i], npv_f = npv_f, 
                                                    wells = optimized_well_notjoint[i], invalid_points = invalid_points) for i in range(count_run)]))
      
      print(f'{not joint}:', npv_joint_mean_rate_notjoint)

    npv_joint_mean.append(npv_joint_mean_rate)
    npv_joint_mean_notjoint.append(npv_joint_mean_rate_notjoint)


  print(f'{joint}:', npv_joint_mean)
  print(f'{not joint}:', npv_joint_mean_notjoint)

Implementation of experiments of the naive approach with GA

In [ ]:
def naive_approach(count_well = 5, count_run = 5, with_opimization_parameter = False, max_iter = 200):
    if with_opimization_parameter:
        mp_list = [0.05 + 0.05 * i for i in range(19)]
        cp_list = [0.05 + 0.05 * i for i in range(19)]
    else: 
        mp_list = [0.55]
        cp_list = [0.95]
    dict_result = {}
    best_result = 0
    best_param = (0.0, 0.0)
    for mp in mp_list:
        for cp in cp_list:
            result_list = []
            if not with_opimization_parameter: 
                result_plot = []
            for run_ in range(count_run):
                print(f'Experiment (No. {run_} of {count_run}) for naive approach with mp: {mp}, cp: {cp}')
                result_one_plot = []
                parameters_for_GA_well = {'max_num_iteration': max_iter // 20,
                                        'population_size_well': 50,
                                        'mutation_probability': 0.6,
                                        'crossover_probability': 0.9, 
                                        'parents_portion': 0.3,
                                        'elit_ratio': 0.01,
                                        'r_mut': 2,
                                        'crossover_type':'uniform'}
                
                parameters_for_well = {'count_well': count_well, 
                                          'r': 2}
                opt_wells = Optimization_alg(field, parameters_for_well, parameters_for_GA_well, npv_f, invalid_points = [[], []])
                pop_well = None 
                for _ in range(20):
                
                  pop_well, optimized_well = opt_wells.run(road_structure = None, init_pop = pop_well)
                  coord_wells = [[well.start[0] for well in optimized_well], [well.start[1] for well in optimized_well]]
                  result_one_plot.append(npv_through_wells(coord_wells = coord_wells, wells = optimized_well))
                print('NPV naive approch:', npv_through_wells(coord_wells = coord_wells, wells = optimized_well))
                result_plot.append(result_one_plot)
                result_list.append(npv_through_wells(coord_wells = coord_wells, wells = optimized_well))
            dict_result[(mp, cp)] = result_list
            if best_result < np.mean(result_list):
                best_result = np.mean(result_list)
                best_param = (mp, cp)
            print(f'NPV results for ({mp, cp}): {np.mean(result_list)}, best result: {best_result} {best_param}')
    if with_opimization_parameter:
      return dict_result
    else:
      return result_plot

A naive PSO-based approach

In [ ]:
from pso import PSO

def naive_approach_PSO(count_well = 5, count_run = 5, with_opimization_parameter = False, max_iter = 200):
    if with_opimization_parameter:
        c_list = [0.5 + 0.1 * i for i in range(20)]
        w_list = [0.05 + 0.05 * i for i in range(19)]
    else: 
        c_list = [1.193]
        w_list = [0.721]
    dict_result = {}
    best_result = 0
    best_param = (0.0, 0.0)
    for c in c_list:
        for w in w_list:
            result_list = []
            if not with_opimization_parameter: 
                result_plot = []
            for run_ in range(count_run):
                print(f'Experiment (No. {run_} of {count_run}) for naive approach with c: {c}, w: {w}')
                result_one_plot = []
                parameters_for_PSO_well = {'max_num_iteration': max_iter // 20,
                                        'population_size_well': 50,
                                        'c0': c,
                                        'c1': c, 
                                        'w': w}
                
                parameters_for_well = {'count_well': count_well, 
                                          'r': 2}
                
                pop_well = None 
                optimized_well = None
                for _ in range(20):
                    pso_algo = PSO(field, parameters_for_PSO_well['population_size_well'],
                               parameters_for_well, parameters_for_PSO_well['c0'],
                               parameters_for_PSO_well['c1'], parameters_for_PSO_well['c1'], pop = pop_well, npv = npv_f, best = optimized_well)
                    
                    pop_well, optimized_well = pso_algo.optimize(parameters_for_PSO_well['max_num_iteration'])
                    coord_wells = [[well.start[0] for well in optimized_well], [well.start[1] for well in optimized_well]]
                    result_one_plot.append(npv_through_wells(coord_wells = coord_wells, wells = optimized_well))
                print('NPV naive approch:', npv_through_wells(coord_wells = coord_wells, wells = optimized_well))
                if not with_opimization_parameter: 
                    result_plot.append(result_one_plot)
                result_list.append(npv_through_wells(coord_wells = coord_wells, wells = optimized_well))
            dict_result[(c, w)] = result_list
            if best_result < np.mean(result_list):
                best_result = np.mean(result_list)
                best_param = (c, w)
            print(f'NPV results for ({c, w}): {np.mean(result_list)}, best result: {best_result} {best_param}')
    if with_opimization_parameter:
      return dict_result
    else:
      return result_plot

Implementing experiments for a consistent and collaborative approach

In [ ]:
def dist_to_extreme_point(extreme_point, point):
    dist = (extreme_point[0] - point._x) ** 2 + (extreme_point[1] - point._y) ** 2
    return np.sqrt(dist)
field = Field(data = data, impossible_points = [[],[]])  
coord_wells = [[], []]

# Loss function for this task
is_closed = False
geometry = Geometry2D(is_closed=is_closed)
def len_num_ration(struct: Structure, start_point = (0, 0), end_point = (20, 20), coord_wells = coord_wells, invalid_points = [[], []]):
    l = []
    p = []
    for poly in struct.polygons:
        l.append(geometry.get_length(poly))
        p.append(poly.points)
    length = sum(l)
    
    ratio = length 
    ratio += dist_to_extreme_point(start_point, p[0][0]) + dist_to_extreme_point(end_point, p[0][-1])
    for i in range(len(invalid_points[0])):
        if distance(Point(invalid_points[0][i], invalid_points[1][i]), struct, geometry) <= 0.5:
            ratio += 4
    for i in range(len(coord_wells[0])):
        ratio += distance(Point(coord_wells[0][i], coord_wells[1][i]), struct, geometry)
  
    return ratio * 3000


is_closed = False
geometry = Geometry2D(is_closed=is_closed)
domain = Domain(allowed_area=[(0, 0),
                              (0, 20),
                              (20, 20),
                              (20, 0),
                              (0, 0)],
                geometry=geometry,
                max_poly_num=1,
                min_poly_num=1,
                max_points_num=20,
                min_points_num=9,
                is_closed=is_closed)

task_setup = Setup(domain=domain)
npv_f = NPV(field, geometry = geometry)
def algo(joint = True, count_well = 5, count_run = 5, invalid_points = [[], []]):
  field = Field(data = data, impossible_points = invalid_points)  
  # Start optimization
  start = timeit.default_timer()

  pop = None
  parameters_for_GA_well = {'max_num_iteration': 20,
                          'population_size_well': 50,
                          'mutation_probability': 0.55,
                          'crossover_probability': 0.95, 
                          'parents_portion': 0.3,
                          'elit_ratio': 0.01,
                          'r_mut': 2,
                          'crossover_type':'uniform'}

  parameters_for_well = {'count_well': count_well, 
                        'r': 2}

  count_run = count_run
  history = []
  history_road = []
  optimized_well_ = []
  optimized_road_ = []

  for run_ in range(count_run):
    print(f'Experiment (No. {run_} of {count_run}) with size_invalid_points: {len(invalid_points[0])}')
    opt_wells = Optimization_alg(field, parameters_for_well, parameters_for_GA_well, npv_f, invalid_points)
    pop_well = None
    optimized_structure = None

    history_npv_for_well = []
    history_npv_for_road = []
    if joint:
      for iter_ in range(30):
          print(iter_)
          pop_well, optimized_well = opt_wells.run(road_structure = optimized_structure, init_pop = pop_well)
          coord_wells = [[well.start[0] for well in optimized_well], [well.start[1] for well in optimized_well]]
          history_npv_for_well.append(-npv_f.calculated_NPV(optimized_well, optimized_structure))
          

      
      #for _ in range(20):
          pop, optimized_structure = optimize(task_setup=task_setup,
                                        objective_function=len_num_ration,
                                        pop_size=50,
                                        max_gens=10, init_pop = pop, invalid_points = invalid_points, coord_wells = coord_wells)
          
          spend_time = timeit.default_timer() - start
          if iter_ == 29:
            # Visualize optimized structure
            visualiser = StructVizualizer(task_setup.domain)
            plt.figure(figsize=(7, 7))

            info = {'spend_time': spend_time,
                    'fitness': len_num_ration(optimized_structure, coord_wells = coord_wells, invalid_points = invalid_points),
                    'type': 'road'}
            visualiser.plot_structure(optimized_structure, info, start_point = (0, 0), end_point = (20, 20), coord_wells = coord_wells, invalid_points = invalid_points)
            
            plt.show()
          history_npv_for_road.append(len_num_ration(optimized_structure, coord_wells = coord_wells, invalid_points = invalid_points))
    else:
      for iter_ in range(20):
          print(iter_)
          pop_well, optimized_well = opt_wells.run(road_structure = optimized_structure, init_pop = pop_well)
          coord_wells = [[well.start[0] for well in optimized_well], [well.start[1] for well in optimized_well]]
          history_npv_for_well.append(-npv_f.calculated_NPV(optimized_well, optimized_structure))
          

      
      for iter_ in range(20):
          print(iter_)
          pop, optimized_structure = optimize(task_setup=task_setup,
                                        objective_function=len_num_ration,
                                        pop_size=50,
                                        max_gens=10, init_pop = pop, invalid_points = invalid_points, coord_wells = coord_wells)
          
          spend_time = timeit.default_timer() - start
          if iter_== 19:
            # Visualize optimized structure
            visualiser = StructVizualizer(task_setup.domain)
            plt.figure(figsize=(7, 7))

            info = {'spend_time': spend_time,
                    'fitness': len_num_ration(optimized_structure, coord_wells = coord_wells, invalid_points = invalid_points),
                    'type': 'road'}
            visualiser.plot_structure(optimized_structure, info, start_point = (0, 0), end_point = (20, 20), coord_wells = coord_wells, invalid_points = invalid_points)
            
            plt.show()
          history_npv_for_road.append(len_num_ration(optimized_structure, coord_wells = coord_wells, invalid_points = invalid_points))
    optimized_well_.append(optimized_well)
    optimized_road_.append(optimized_structure)

    history.append(history_npv_for_well)
    history_road.append(history_npv_for_road)
  return optimized_well_, optimized_road_, history, history_road

def algo_PSO(joint = True, count_well = 5, count_run = 5, invalid_points = [[], []]):
  # Start optimization
  start = timeit.default_timer()
  field = Field(data = data, impossible_points = invalid_points)  
  pop = None
  parameters_for_PSO_well = {'max_num_iteration': 10,
                                        'population_size_well': 50,
                                        'c0': 1.193,
                                        'c1': 1.193, 
                                        'w': 0.721}

  parameters_for_well = {'count_well': count_well, 
                        'r': 2}

  count_run = count_run
  history = []
  history_road = []
  optimized_well_ = []
  optimized_road_ = []

  for run_ in range(count_run):
    print(f'Experiment (No. {run_} of {count_run}) with size_invalid_points: {len(invalid_points[0])}')
    
    pop_well = None
    optimized_well = None
    optimized_structure = None

    history_npv_for_well = []
    history_npv_for_road = []
    if joint:
      for iter_ in range(30):
          print(iter_)
          pso_algo = PSO(field, parameters_for_PSO_well['population_size_well'],
                        parameters_for_well, parameters_for_PSO_well['c0'],
                        parameters_for_PSO_well['c1'], parameters_for_PSO_well['c1'], pop = pop_well, npv = npv_f, best = optimized_well)
                    
          pop_well, optimized_well = pso_algo.optimize(parameters_for_PSO_well['max_num_iteration'])
          coord_wells = [[well.start[0] for well in optimized_well], [well.start[1] for well in optimized_well]]
          history_npv_for_well.append(-npv_f.calculated_NPV(optimized_well, optimized_structure))
          

      
      #for _ in range(20):
          pop, optimized_structure = optimize(task_setup=task_setup,
                                        objective_function=len_num_ration,
                                        pop_size=50,
                                        max_gens=10, init_pop = pop, invalid_points = invalid_points, coord_wells = coord_wells)
          
          spend_time = timeit.default_timer() - start
          if iter_ == 29:
            # Visualize optimized structure
            visualiser = StructVizualizer(task_setup.domain)
            plt.figure(figsize=(7, 7))

            info = {'spend_time': spend_time,
                    'fitness': len_num_ration(optimized_structure, coord_wells = coord_wells, invalid_points = invalid_points),
                    'type': 'road'}
            visualiser.plot_structure(optimized_structure, info, start_point = (0, 0), end_point = (20, 20), coord_wells = coord_wells, invalid_points = invalid_points)
            
            plt.show()
          history_npv_for_road.append(len_num_ration(optimized_structure, coord_wells = coord_wells, invalid_points = invalid_points))
    else:
      for iter_ in range(20):
          print(iter_)
          pso_algo = PSO(field, parameters_for_PSO_well['population_size_well'],
                        parameters_for_well, parameters_for_PSO_well['c0'],
                        parameters_for_PSO_well['c1'], parameters_for_PSO_well['c1'], pop = pop_well, npv = npv_f, best = optimized_well)
                    
          pop_well, optimized_well = pso_algo.optimize(parameters_for_PSO_well['max_num_iteration'])
          coord_wells = [[well.start[0] for well in optimized_well], [well.start[1] for well in optimized_well]]
          history_npv_for_well.append(-npv_f.calculated_NPV(optimized_well, optimized_structure))
          

      
      for iter_ in range(20):
          print(iter_)
          pop, optimized_structure = optimize(task_setup=task_setup,
                                        objective_function=len_num_ration,
                                        pop_size=50,
                                        max_gens=10, init_pop = pop, invalid_points = invalid_points, coord_wells = coord_wells)
          
          spend_time = timeit.default_timer() - start
          if iter_== 19:
            # Visualize optimized structure
            visualiser = StructVizualizer(task_setup.domain)
            plt.figure(figsize=(7, 7))

            info = {'spend_time': spend_time,
                    'fitness': len_num_ration(optimized_structure, coord_wells = coord_wells, invalid_points = invalid_points),
                    'type': 'road'}
            visualiser.plot_structure(optimized_structure, info, start_point = (0, 0), end_point = (20, 20), coord_wells = coord_wells, invalid_points = invalid_points)
            
            plt.show()
          history_npv_for_road.append(len_num_ration(optimized_structure, coord_wells = coord_wells, invalid_points = invalid_points))
    optimized_well_.append(optimized_well)
    optimized_road_.append(optimized_structure)

    history.append(history_npv_for_well)
    history_road.append(history_npv_for_road)
  return optimized_well_, optimized_road_, history, history_road

Application of experiments

In [ ]:
step = 15
naive_approach = [[dict_res[j][i] for j in range(len(dict_res))] 
                                       for i in range(len(dict_res[0]))]

flierprops = dict(marker='.', 
                  linestyle='none')
plt.boxplot(naive_approach, meanline = True, flierprops = flierprops)
plt.plot(range(1, len(naive_approach) + 1), 
          np.mean(naive_approach, axis = 1), label = 'Average NPV roads value for 10 runs')
plt.xticks(range(1, len(naive_approach) + 1), [step * i + step for i in range(len(naive_approach))], 
                 rotation=90)
plt.xlabel('Iteration number')
plt.ylabel('NPV values')
plt.legend()
plt.title(f'Learning curve for naive approach for 7 wells')
plt.tight_layout()
plt.savefig('NA_PSO_7_3000_lc.png')


TypeError: ignored

In [ ]:
x = [0.5 + 0.1 * i for i in range(20)]
y = [0.05 + 0.05 * i for i in range(19)]
x, y =np.meshgrid(x, y) 

z = np.zeros(x.shape)
for i in range(x.shape[0]):
  for j in range(x.shape[1]):
    key = (x[i][j], y[i, j])
    z[i, j] = np.mean(dict_res[key]) 

In [ ]:
x = np.linspace(0.5, 2.5, 20)
y = np.linspace(0.05, 0.95, 19)
X, Y = np.meshgrid(x, y)

contours = plt.contourf(X, Y, z, 20, cmap = 'YlOrRd')
plt.xlabel('Parameter c_1, c_2')
plt.ylabel('Parameter w')
plt.title('Optimization of PSO hyperparameters\n Max NPV values with parameters (с=1.193, cp = 0.721)')

plt.colorbar();
plt.savefig('PSO_hyper.png')

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.contour(x, y, z,
                linewidths=np.arange(.5, 4, .5),
                colors=('r', 'green', 'blue', (1, 1, 0), '#afeeee', '0.5'),
                )
ax.set_xlabel('Probability of mutation')
ax.set_ylabel('Crossover probability')
ax.set_zlabel('NPV values')
plt.title('Optimization of GA hyperparameters\n Max NPV values with parameters\n (mp=0.6, cp = 0.9)')
plt.savefig('100.png')

In [ ]:
225426.0421834106/222608.44005457303

In [ ]:
flierprops = dict(marker='.', 
                  linestyle='none')
plt.boxplot(history_boxplot, meanline = True, flierprops = flierprops)
plt.plot(range(1, 51), np.mean(history_boxplot, axis = 1), label = 'Average NPV value for 5 runs')
plt.xticks(range(1, 51, 2), [10 * i * 2 + 10 for i in range(50)], rotation=90)
plt.xlabel('Iteration number')
plt.ylabel('NPV values')
plt.legend()
plt.title('Learning curve for optimizing the location of 5 wells \n with considering and optimizing roads')
plt.show()

In [ ]:
history_boxplot_road = [[history_road[j][i] for j in range(len(history_road))] for i in range(len(history_road[0]))]

In [ ]:
flierprops = dict(marker='.', 
                  linestyle='none')
plt.boxplot(history_boxplot_road, meanline = True, flierprops = flierprops)
plt.plot(range(1, 81), np.mean(history_boxplot_road, axis = 1), label = 'Average NPV roads value for 5 runs')
plt.xticks(range(1, 81), [10 * i + 10 for i in range(80)], rotation=90)
plt.xlabel('Iteration number')
plt.ylabel('NPV roads values')
plt.legend()
plt.title('Learning curve for optimizing the location of roads \n with considering and optimizing 5 wells')
plt.show()

In [ ]:
np.mean([npv_joint(optimized_road_[i], npv_f = npv_f, wells = optimized_well_[i]) for i in range(3)])

In [ ]:
print(npv_f.calculated_NPV(optimized_well))

In [ ]:
distance(Point(5, 6), optimized_structure, geometry)

In [ ]:
plt.plot(range(0, 4),[3.21, 4.66, 8.73, 23.17], label = '3 wells')
plt.plot(range(0, 4),[1.93, 2.66, 5.35, 24.38], label = '5 wells')
plt.plot(range(0, 4),[2.48, 7.26, 4.49, 17.38], label = '7 wells')
plt.xticks(range(0, 4), [500, 1000, 3000, 4000])
plt.xlabel('Road cost coefficient')
plt.ylabel('Values of the K coefficient')
plt.legend()

In [ ]:
plt.plot(range(0, 4),[1.16, 6.14, 12.13, 21.78], label = '3 wells')
plt.plot(range(0, 4),[1.43, 4.29, 6.83, 15.03], label = '5 wells')
plt.plot(range(0, 4),[2.99, 3.54, 8.35, 16.85], label = '7 wells')
plt.xticks(range(0, 4), [500, 1000, 3000, 4000])
plt.xlabel('Road cost coefficient')
plt.ylabel('Values of the K coefficient')
plt.legend()